# Quien es quien en los precios 


Data Analysis on Profeco Prices: 

- https://es.wikipedia.org/wiki/Procuradur%C3%ADa_Federal_del_Consumidor
    

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType
from pyspark.sql.functions import col,asc,desc,rank, isnan, when, count, to_timestamp, year,month, translate
from pyspark.sql.window import Window
import pyspark.sql.functions as f
import pandas as pd
import seaborn as sns
import folium

In [4]:
pd.set_option('display.max_rows', 500)
pd.options.display.max_columns


20

In [5]:
spark = SparkSession.builder \
     .master("local") \
     .appName("analisis_precios") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

In [6]:
df = spark.read.csv('all_data.csv', header=True, inferSchema=True)

In [4]:
df.show(5) #(vertical = True)

+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|            producto|        presentacion|   marca|       categoria|        catalogo|precio|       fechaRegistro|   cadenaComercial|      giro|     nombreComercial|           direccion|          estado|           municipio| latitud|  longitud|
+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|CUADERNO FORMA IT...|96 HOJAS PASTA DU...|ESTRELLA|MATERIAL ESCOLAR|UTILES ESCOLARES|  25.9|2011-05-18 00:00:...|ABASTECEDORA LUMEN|PAPELERIAS|ABASTECEDORA LUME...|CANNES No. 6 ESQ....|DISTRITO FEDERAL|TLALPAN          ...|19.29699|-99.125417|
|            CRAYONE

In [5]:
df = df.withColumn('fechaRegistro', df['fechaRegistro'].cast(DateType()))

In [6]:
df.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: double (nullable = true)
 |-- fechaRegistro: date (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



# 1. Procesamiento de los datos
## a. ¿Cuántos registros hay?

In [5]:
df.count() # 62530715

62530715

## b. ¿Cuántas categorías?

In [9]:
df.select('categoria').distinct().count() #42

42

## c. ¿Cuántas cadenas comerciales están siendo monitoreadas?

In [11]:
df.select('cadenaComercial').distinct().count()

706

## d. ¿Cómo podrías determinar la calidad de los datos? ¿Detectaste algún tipo de inconsistencia o error en la fuente?

 - Hay datos que tienen Nulo en el estado o en el municipio. esos datos no me sirven para el tipo de calculos que debo hacer. 
 - No los he eliminado porque es muy caro computacionalmente hacerlo. 
 - Asimismo, en la variable municipio, observo que hay municipios que estan ingresados escirtos de diferentes maneras. Esto me generara problemas de manera posterior, al momento de querer hacer mi canasta de precios por ciudad ya que se me generara mas de un indice por ciudad. Sin embargo considero que esto no altera enormemente los resultados finales (aunque deberia ser corregido posteriormente). 

## e. ¿Cuáles son los productos más monitoreados en cada entidad?

In [26]:
window = Window.partitionBy('estado').orderBy(col('count').desc())

In [41]:
product_top10_state = df.select('estado','producto')\
.groupby('estado','producto').count()\
.orderBy(col('estado').asc(),col('count').desc())\
.select('*', rank().over(window).alias('rank'))\
.filter(col('rank')<=10)\
.drop('rank')#\
#.show(truncate=False)

product_top10_state.toPandas()

,estado,producto,count
0,QUINTANA ROO,FUD,34846
1,QUINTANA ROO,REFRESCO,34367
2,QUINTANA ROO,LAVADORAS,32347
3,QUINTANA ROO,LECHE ULTRAPASTEURIZADA,31155
4,QUINTANA ROO,DETERGENTE P/ROPA,28691
...,...,...,...
338,MÉXICO,YOGHURT,94852
339,MÉXICO,MAYONESA,94286
340,MÉXICO,CHILES EN LATA,92539
341,MÉXICO,SHAMPOO,92307


## f. ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

Soriana es la cadena con mayor variedad, con 1059 productos

In [42]:
top_variedad_cadena = df.select('cadenaComercial','producto')\
.groupby('cadenaComercial')\
.agg(f.countDistinct('producto'))\
.orderBy(col('count(producto)').desc())
top_variedad_cadena.toPandas().head(10)

,cadenaComercial,count(producto)
0,SORIANA,1059
1,WAL-MART,1051
2,MEGA COMERCIAL MEXICANA,1049
3,COMERCIAL MEXICANA,1036
4,CHEDRAUI,1026
5,MERCADO SORIANA,1024
6,BODEGA AURRERA,1012
7,HIPERMERCADO SORIANA,1006
8,H.E.B.,1001
9,SORIANA PLUS,999


# 2. Análisis exploratorio

## a.Genera una canasta de productos básicos que te permita comparar los precios geográfica y temporalmente. Justifica tu elección y procedimiento

Dado que el dataset es muy grande, tengo que encontrar una estrategia de reducirlo un poco. 
Mi objetivo es hacer una canasta de productos basicos. 
Para ello requiero elegir productos esenciales pero que a su vez sean representativos. 
Por lo que sigo la siguiente estratengia:
1. eligo los 400 productos que mas veces aparecen en el dataframe
2. Eligo una canasta de productos basicos a partir de esa lista (y teniendo en cuenta los productos que son esenciales siguiendo esta fuente: 
https://www.gob.mx/canastabasica
3. Genero un nuevo data set que solo contenga esos productos 

Utilizando el siguiente codigo, generé una lista de 12 productos esenciales a partir de los 400 que más aparecen en el dataset: 

In [ ]:
#No corro este codigo porque genera una salida muy larga: 
#df_productos = df.select('producto')\
#.groupby('producto').count()\
#.orderBy(col('count').desc())
#df_productos.limit(400).toPandas().head(400)

Lista de productos elegidos: 

In [7]:
canasta = ['HARINA DE MAIZ', 'ARROZ', 'LECHE ULTRAPASTEURIZADA', 'AZUCAR', 'ACEITE', 'TOALLA FEMENINA', 'JABON DE TOCADOR', 'AVENA', 'CHILES EN LATA', 'HARINA DE TRIGO', 'PASTA PARA SOPA', 'LENTEJA']

In [8]:
df_canasta = df.filter(df.producto.isin(canasta))
df_canasta.count()

4558147

Para elegir con qué presentacion quedarme de cada producto, me quedo con la que más veces aparece de cada uno.

In [11]:
#No corro el codigo porque genera una salida muy larga: 
#canasta_presentaciones= df_canasta.select('producto','presentacion')\
#.groupby('producto','presentacion').count()\
#.orderBy(col('producto').asc(),col('count').desc())
#canasta_presentaciones.toPandas()

Estas son las presentaciones elegidas de los 12 productos (y filtro por presentaciones): 

In [9]:
presentaciones_elegidas = ['PARCIALMENTE DESCREMADA. CAJA 1 LT.','BOLSA 500 GR.' ,'BOTELLA 1 LT. MIXTO', 'BOLSA 1 KG. SUPER EXTRA', 'BOTE 510 GR. HOJUELA', 'BOLSA PLASTICO 2 KG. ESTANDAR MORENA','LATA 220 GR. JALAPENOS RAJAS','PAQUETE 1 KG.','PAQUETE 1 KG.','BARRA 180 GR.','PAQUETE 200 GR. SPAGHETTI','BOLSA 10 PIEZAS. NATURAL FLEX. NOCTURNA. CON ALAS'  ]


In [10]:
df_canasta_presentaciones = df_canasta.filter(df.presentacion.isin(presentaciones_elegidas))
df_canasta_presentaciones.count()

1477821

## b. ¿Cuál es la ciudad más cara del país? ¿Cuál es la más barata?

In [15]:
costo_ciudad = df_canasta_presentaciones.select('producto','precio','municipio')\
.groupby('municipio','producto')\
.avg('precio')

In [16]:
costo_ciudad_2 = costo_ciudad.toPandas()\
.pivot(index='municipio', columns='producto', values='avg(precio)')

Reemplazamos nas con el promedio del valor del producto:

Reemplazo los casos en los que hay nas con la media de la columna 
(esto se justifica porque si no tengo valor sobre un producto en un municipio, lo puedo rellenar con el valor medio de ese producto entre todos los municipios, para que ese valor no altere la relacion del indice de ese municipio con respecto a todos los demas en el promedio final)

In [17]:
costo_ciudad_2.isna().sum()

producto
ACEITE                     0
ARROZ                      4
AVENA                      2
AZUCAR                     3
CHILES EN LATA             0
HARINA DE MAIZ             1
HARINA DE TRIGO            0
JABON DE TOCADOR           0
LECHE ULTRAPASTEURIZADA    1
LENTEJA                    2
PASTA PARA SOPA            0
TOALLA FEMENINA            3
dtype: int64

In [76]:
na_list = ['ARROZ', 'AVENA', 'AZUCAR','HARINA DE MAIZ',  'TOALLA FEMENINA']

In [80]:
[costo_ciudad_2[i].fillna(value=costo_ciudad_2[i].mean(), inplace=True) for i in na_list]

[None, None, None, None, None]

In [77]:
#for i in na_list:
#    costo_ciudad_2[i].fillna(value=costo_ciudad_2[i].mean(), inplace=True)


In [81]:
costo_ciudad_2['costo_ciudad'] = costo_ciudad_2.mean(axis=1)

### La ciudad mas barate del pais es:

In [92]:
costo_ciudad_2['costo_ciudad'].to_frame().sort_values('costo_ciudad').iloc[0:1]

,costo_ciudad
municipio,
CARDENAS,14.675806


### La ciudad mas cara es Xalapa (y la colonia magdalena contreras de la ciudad de mexico):

In [103]:
costo_ciudad_2['costo_ciudad'].to_frame().sort_values('costo_ciudad').tail(2)

,costo_ciudad
municipio,
XALAPA,17.078664
MAGDALENA CONTRERAS,17.629211


# c. ¿Hay algún patrón estacional entre años?

In [116]:
costo_anual = df_canasta_presentaciones.select('producto','precio','fechaRegistro')\
.groupBy(year('fechaRegistro').alias('year'), 'producto')\
.avg('precio')

In [123]:
costo_anual_2 = costo_anual.groupby('year')\
.avg('avg(precio)')\
.orderBy('year')

In [124]:
costo_anual_pd = costo_anual_2.toPandas()

In [133]:
costo_anual_pd.rename(columns={"avg(avg(precio))": "costo_vida_por_anio"}, inplace = True)

La siguiente tabla muestra el costo de vida para cada año. No parece haber un patrón, como por ejemplo un aumento constante de los precios año tras año: 

In [134]:
#No es el costo de vida por anio si no el costo de vida de ese anio:
costo_anual_pd

,year,costo_vida_por_anio
0,2011,15.129952
1,2012,16.312728
2,2013,15.637632
3,2014,15.315976
4,2015,15.724572
5,2016,16.767626


## d. ¿Cuál es el estado más caro y en qué mes?

In [18]:
costo_mensual = df_canasta_presentaciones.select('producto','precio','estado','fechaRegistro')\
.groupBy('estado', month('fechaRegistro').alias('month'), 'producto')\
.avg('precio')

In [19]:
window_2 = Window.partitionBy('estado', 'month').orderBy( col('avg(avg(precio))').desc() )

In [20]:
costo_mensual_2 = costo_mensual.groupby('estado','month')\
.avg('avg(precio)')\
.orderBy(col('avg(avg(precio))').desc())\
.select('*', rank().over(window_2).alias('rank'))\
.filter(col('rank')<=1)\
.drop('rank')

In [21]:
costo_mensual_pd = costo_mensual_2.toPandas()
costo_mensual_pd.rename(columns={"avg(avg(precio))": "costo_vida_mes"}, inplace = True)

### El estado mas caro es Guerrero en el mes de abril:

In [24]:
costo_mensual_pd.sort_values('costo_vida_mes',ascending=False).head(1)

,estado,month,costo_vida_mes
355,GUERRERO,4,16.597765


## e. ¿Cuáles son los principales riesgos de hacer análisis de series de tiempo conestos datos?

El principal riesgo es que no hay una cantidad similar de datos en todos los períodos si no que hay una cantidad mucho mayor de datos en los períodos más recientes. 

# 3. Visualización
## a. Genera un mapa que nos permita identificar la oferta de categorías en la zona metropolitana de León Guanajuato y el nivel de precios en cada una de ellas. Se darán puntos extra si el mapa es interactivo

In [11]:
leon = ['LEON                                    ','LEÓN']

In [12]:
df_leon = df_canasta_presentaciones.filter(df_canasta_presentaciones.municipio.isin(leon))
df_leon.count()

52085

In [13]:
pd_leon = df_leon.toPandas()

In [14]:
pd_leon.rename(columns={'LEON                                    ': "LEON"}, inplace = True)
pd_leon.rename(columns={'LEÓN': "LEON"}, inplace = True)

In [15]:
pd_leon['lat_long']= pd_leon['latitud']+pd_leon['longitud']

In [30]:
pd_leon['categoria'] = pd_leon['categoria'].dropna()

In [ ]:
df.mask(df.eq('None')).dropna()

In [22]:
import numpy as np

In [36]:
pd_leon.categoria.fillna(value=' ', inplace=True)

In [37]:
pd_leon['categoria'].unique()

array(['FRUTAS Y LEGUMBRES PROCESADAS',
       'GALLETAS PASTAS Y HARINAS DE TRIGO',
       'TORTILLAS Y DERIVADOS DEL MAIZ', 'ARROZ Y CEREALES PREPARADOS',
       'LECHE PROCESADA', 'AZUCAR', 'ACEITES Y GRASAS VEG. COMESTIBLES',
       'ARTS. PARA EL CUIDADO PERSONAL',
       'ARTS. DE PAPEL P/HIGIENE PERSONAL', 'LEGUMBRES SECAS', '_'],
      dtype=object)

In [ ]:
df['a'] = df['a'].apply(lambda x: x + 1)

In [108]:
pd_leon['all_categories'] = pd_leon.groupby(['lat_long'])['categoria'].transform(lambda x : ', <br>'.join(x))

In [55]:
pd_leon.shape

(52085, 17)

In [60]:
pd_leon['lat_long'].value_counts()

21.161521-101.691985    5756
21.132543-101.659225    5433
21.103839-101.693575    5163
21.095156-101.62957     5009
21.107162-101.660872    4912
21.099966-101.670745    4690
21.141389-101.687397    3954
21.139468-101.6871      3167
21.124324-101.685945    3136
21.146492-101.684757    3125
21.148644-101.704063    2442
21.102498-101.68722     1870
21.125658-101.68559     1193
21.122775-101.685398    1054
21.130531-101.683038     885
21.095156-101.629570     130
21.139468-101.687100      88
21.102498-101.687220      78
Name: lat_long, dtype: int64

In [99]:
pd_leon_ok = pd_leon.groupby(['lat_long','all_categories','latitud','longitud']).count().reset_index()

In [65]:
pd_leon_ok

,lat_long,all_categories,latitud,longitud,producto,presentacion,marca,categoria,catalogo,precio,fechaRegistro,cadenaComercial,giro,nombreComercial,direccion,estado,municipio
0,21.095156-101.62957,"GALLETAS PASTAS Y HARINAS DE TRIGO, GALLETAS P...",21.095156,-101.62957,5009,5009,5009,5009,5009,5009,5009,5009,5009,5009,5009,5009,5009
1,21.095156-101.629570,"ARTS. DE PAPEL P/HIGIENE PERSONAL, TORTILLAS Y...",21.095156,-101.629570,130,130,130,130,130,130,130,130,130,130,130,130,130
2,21.099966-101.670745,"ARTS. PARA EL CUIDADO PERSONAL, GALLETAS PASTA...",21.099966,-101.670745,4690,4690,4690,4690,4690,4690,4690,4690,4690,4690,4690,4690,4690
3,21.102498-101.68722,"ACEITES Y GRASAS VEG. COMESTIBLES, ACEITES Y G...",21.102498,-101.68722,1870,1870,1870,1870,1870,1870,1870,1870,1870,1870,1870,1870,1870
4,21.102498-101.687220,"ACEITES Y GRASAS VEG. COMESTIBLES, ARROZ Y CER...",21.102498,-101.687220,78,78,78,78,78,78,78,78,78,78,78,78,78
5,21.103839-101.693575,"ARTS. PARA EL CUIDADO PERSONAL, FRUTAS Y LEGUM...",21.103839,-101.693575,5163,5163,5163,5163,5163,5163,5163,5163,5163,5163,5163,5163,5163
6,21.107162-101.660872,"GALLETAS PASTAS Y HARINAS DE TRIGO, GALLETAS P...",21.107162,-101.660872,4912,4912,4912,4912,4912,4912,4912,4912,4912,4912,4912,4912,4912
7,21.122775-101.685398,"FRUTAS Y LEGUMBRES PROCESADAS, GALLETAS PASTAS...",21.122775,-101.685398,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054
8,21.124324-101.685945,"TORTILLAS Y DERIVADOS DEL MAIZ, ARROZ Y CEREAL...",21.124324,-101.685945,3136,3136,3136,3136,3136,3136,3136,3136,3136,3136,3136,3136,3136
9,21.125658-101.68559,"LECHE PROCESADA, AZUCAR, FRUTAS Y LEGUMBRES PR...",21.125658,-101.68559,1193,1193,1193,1193,1193,1193,1193,1193,1193,1193,1193,1193,1193


## Creo el mapa

In [100]:
m = folium.Map(location=[21.122775, -101.685398],
               tiles="CartoDB positron",
               zoom_start=11)

In [101]:
for index, row in pd_leon_ok.iterrows():
    tooltip = f"categories:<br> {row['all_categories']}"
    
    folium.Circle(
        location=[row['latitud'],row['longitud']],
        tooltip = tooltip,
        radius = 200,
        color = "red",
        fill = True,
        opacity = 0.8
    ).add_to(m)


In [102]:
m